In [3]:
# pip install imblearn

     |████████████████████████████████| 194kB 7.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import string
import warnings
import math
import nltk
import json
from langdetect import detect
from libsvm import *
from nltk.stem import PorterStemmer
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler

from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
import dask.dataframe as dd
import multiprocessing
import swifter
warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('words')
from spellchecker import SpellChecker
from imblearn.over_sampling import SMOTE

spell = SpellChecker()

# pd.set_option('display.max_colwidth', None)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anishajauhari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/anishajauhari/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [31]:
#### Loading Target Values 
target = pd.read_csv("../../data/target.csv")

In [41]:
target = target.drop(columns = ["Unnamed: 0"])

## Classification 

In [3]:
def convertToSparse(df):
    df_sparse = csr_matrix(df.iloc[:, :], dtype=np.int8).toarray()
    return pd.DataFrame(df_sparse, index = df.index, columns = df.columns[:])

In [4]:
def trainTestSplit(df, y):
    return train_test_split(df, y, test_size=0.20, random_state=42)

In [45]:
def trainTestSplitWithImbalancedClassHandling(df, y):
    ### Applying SMOTE Algorithm ###
    sm = SMOTE(random_state = 2)
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.20, random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    return X_train, X_test, y_train, y_test

In [34]:
def model_fit(X_train, y_train, model = "NB"):
    if model == "NB":
        NB_Classifier = MultinomialNB()
        model = NB_Classifier.fit(X_train, y_train)
    elif model == "SVM":
        model = OneVsRestClassifier(BaggingClassifier(
            SVC(kernel='rbf', probability=True, class_weight='balanced'), 
                max_samples=1.0 / 10, 
                n_estimators=10))
        model.fit(X_train, y_train)
    else:
        rf = RandomForestClassifier(max_depth = 2, random_state = 0)
        model = rf.fit(X_train, y_train)
    return model

In [35]:
def evaluate_model(y_test, y_pred, model_name):
    print("\nConfusion Matrix : "+model_name)
    print(confusion_matrix(y_test, y_pred))
    print("\n*******************************")
    print("\nClassification Report : "+model_name)
    print(classification_report(y_test, y_pred))

In [49]:
def run_model(df):
    convertToSparse(df)
    X_train, X_test, y_train, y_test = trainTestSplitWithImbalancedClassHandling(df, target)
    trans = MinMaxScaler()
    X_train = trans.fit_transform(X_train)
    X_test = trans.fit_transform(X_test)
    model = model_fit(X_train, y_train, "NB")
    y_pred = model.predict(X_test)
    y_pred_changed = pd.Series(y_pred, index = y_test.index)
    evaluate_model(y_test, y_pred, "NB")
    
    print("--- Naive Bayes Misclassification ---")
#     false_neg = X_test[list((y_test == 1) & (y_pred == 0))]
#     false_pos = X_test[list((y_test == 0) & (y_pred == 1))]
#     if len(false_neg) > 0:
#         print("--- False Neg Results ---")
#         print(y_test[false_neg[:5]["index"]])
#         print(y_pred_changed[false_neg[:5]["index"]])
#         display(rock_pop_dataset.loc[false_neg[:5]["index"]])
#     if len(false_pos)>0:
#         print("--- False Pos Results ---")
#         print(y_test[false_pos[:5]["index"]])
#         print(y_pred_changed[false_pos[:5]["index"]])
#         display(rock_pop_dataset.loc[false_pos[:5]["index"]])
    
    
    model = model_fit(X_train, y_train, "RF")
    y_pred = model.predict(X_test)
    y_pred_changed = pd.Series(y_pred, index = y_test.index)
#     print("--- Random Forest Misclassification ---")
#     false_neg = X_test[list((y_test == 1) & (y_pred == 0))]
#     false_pos = X_test[list((y_test == 0) & (y_pred==1))]
#     if len(false_neg) > 0:
#         print("--- False Neg Results ---")
#         print(y_test[false_neg[:5]["index"]])
#         print(y_pred_changed[false_neg[:5]["index"]])
#         display(rock_pop_dataset.iloc[false_neg[:5]["index"]])
#     if len(false_pos)>0:
#         print("--- False Pos Results ---")
#         print(y_test[false_pos[:5]["index"]])
#         print(y_pred_changed[false_pos[:5]["index"]])
#         display(rock_pop_dataset.loc[false_pos[:5]["index"]])
    evaluate_model(y_test, y_pred, "RF")
    
    
    
    model = model_fit(X_train, y_train, "SVM")
    y_pred = model.predict(X_test)
    y_pred_changed = pd.Series(y_pred, index = y_test.index)
#     print("--- SVM Misclassification ---")
#     false_neg = X_test[list((y_test == 1) & (y_pred == 0))]
#     false_pos = X_test[list((y_test == 0) & (y_pred==1))]
#     if len(false_neg) > 0:
#         print("--- False Neg Results ---")
#         print(y_test[false_neg[:5]["index"]])
#         print(y_pred_changed[false_neg[:5]["index"]])
#         display(rock_pop_dataset.loc[false_neg[:5]["index"]])
#     if len(false_pos)>0:
#         print("--- False Pos Results ---")
#         print(y_test[false_pos[:5]["index"]])
#         print(y_pred_changed[false_pos[:5]["index"]])
#         display(rock_pop_dataset.loc[false_pos[:5]["index"]])
    evaluate_model(y_test, y_pred, "SVM")

### Experiment 2 - TFIDF

In [ ]:
features = pd.read_csv("../../data/features_expt_2.csv")
features = features[0:-1]
features.drop(columns=["Unnamed: 0"], inplace = True)
features.shape ### HACK -- Need to fix

In [ ]:
run_model(features)

,0
125028,1
6529,1
74096,0
23735,1
1277,1
...,...
119879,1
103694,0
860,1
15795,1


        Unnamed: 0  misspelled_words  unique_words  slang_words  danceability  \
0           125028                 0            28            2         0.000   
1             6529                 1            44            4         0.551   
2            74096                 0            43            2         0.000   
3            23735                 1            61            4         0.000   
4             1277                 1            39            1         0.000   
...            ...               ...           ...          ...           ...   
150859       86845                 3            77            4         0.000   
150860       26633                 1            92            1         0.000   
150861       59557                 0            31            1         0.000   
150862       25337                 3            78            0         0.000   
150863       57261                 0            19            0         0.000   

        energy  key  loudne

### Experiment 2 - IG

In [ ]:
features_ig = pd.read_csv("../../data/features_expt_2_ig.csv")
features_ig = features_ig[0:-1]
features_ig.drop(columns=["Unnamed: 0"], inplace = True)
features_ig.shape ### HACK -- Need to fix

In [ ]:
run_model(features_ig)

### Analysing TFIDF Models 

In [75]:
run_model(tfidf_df_100)

--- Naive Bayes Misclassification ---
--- False Pos Results ---
24002     0
46011     0
135453    0
1621      0
87593     0
dtype: int8
24002     1
46011     1
135453    1
1621      1
87593     1
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
24002,henry,2010-01-01,emma dean,Pop,Aching to be in the room 'cos no one can tell ...,aching room cos one tell everythings going way...
46011,i need a disguise,2014-01-01,belinda carlisle,Pop,There's a mask on the wall That I should be we...,theres mask wall wearing keep seeing im really...
135453,wow,2006-01-01,brandy,Pop,No more sittin' at home at night Waiting for y...,sittin home night waiting come home cry found ...
1621,the homes of donegal,2009-01-01,celtic thunder,Pop,I just dropped in to see you all I'll only sta...,dropped see ill stay awhile want hear youre ge...
87593,time of year,2006-01-01,ashanti,Pop,Christmas is the time of year That we become a...,christmas time year become one tears joy heart...


--- Random Forest Misclassification ---
--- False Pos Results ---
24002     0
46011     0
135453    0
1621      0
87593     0
dtype: int8
24002     1
46011     1
135453    1
1621      1
87593     1
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
24002,henry,2010-01-01,emma dean,Pop,Aching to be in the room 'cos no one can tell ...,aching room cos one tell everythings going way...
46011,i need a disguise,2014-01-01,belinda carlisle,Pop,There's a mask on the wall That I should be we...,theres mask wall wearing keep seeing im really...
135453,wow,2006-01-01,brandy,Pop,No more sittin' at home at night Waiting for y...,sittin home night waiting come home cry found ...
1621,the homes of donegal,2009-01-01,celtic thunder,Pop,I just dropped in to see you all I'll only sta...,dropped see ill stay awhile want hear youre ge...
87593,time of year,2006-01-01,ashanti,Pop,Christmas is the time of year That we become a...,christmas time year become one tears joy heart...


--- SVM Misclassification ---
--- False Neg Results ---
131846    1
21115     1
14633     1
90017     1
61905     1
dtype: int8
131846    0
21115     0
14633     0
90017     0
61905     0
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
131846,maybe then someday,2005-01-01,abandoned pools,Rock,What's this thing that's haunting me Won't my ...,whats thing thats haunting wont mind let sober...
21115,daughter of light,2007-01-01,carole king,Rock,Daughter of light You're a welcome sight To a ...,daughter light youre welcome sight weary soul ...
14633,baby don t you lie to me,2015-01-01,fratellis,Rock,Well little girl you can roll your eyes At the...,well little girl roll eyes sight everything sp...
90017,ideology,2014-01-01,billy bragg,Rock,When one voice rules the nation Just because t...,one voice rules nation theyre top pile doesnt ...
61905,sorry,2007-01-01,daughtry,Rock,Will you listen to my story? It just be a minu...,listen story minute explain whatever happened ...


--- False Pos Results ---
169862    0
196460    0
168066    0
178700    0
194214    0
dtype: int8
169862    1
196460    1
168066    1
178700    1
194214    1
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
169862,in danger,2010-01-01,aaron fresh,Pop,"Ohh Noo Noo, No ( Verse 1 ) He Told You Lies F...",ohh noo noo verse told lies long everyone s...
196460,brave face,2007-01-01,delta goodrem,Pop,Put your brave face on The one you wore when y...,put brave face one wore stole heart wont hold ...
168066,she works hard for the money,2012-01-01,american idol,Pop,She works hard for the money So hard for it ho...,works hard money hard honey works hard money b...
178700,tender love,2015-01-01,exo,Pop,English Translation: Come to the inner side of...,english translation come inner side sidewalk i...
194214,have fun,2006-01-01,diana ross,Pop,"Hey, everybody Have fun again I want you to ha...",hey everybody fun want fun fun want fun like l...


In [81]:
display(rock_pop_dataset.loc[131846])

song                                              maybe then someday
year                                                      2005-01-01
artist                                               abandoned pools
genre                                                           Rock
lyrics             What's this thing that's haunting me Won't my ...
filtered_lyrics    whats thing thats haunting wont mind let sober...
Name: 131846, dtype: object

In [ ]:
run_model(tfidf_df_200)

In [44]:
run_model(tfidf_df_500)


Confusion Matrix : NB
[[    3  6152]
 [    0 18954]]

*******************************

Classification Report : NB
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      6155
           1       0.75      1.00      0.86     18954

    accuracy                           0.75     25109
   macro avg       0.88      0.50      0.43     25109
weighted avg       0.82      0.75      0.65     25109


Confusion Matrix : RF
[[    0  6155]
 [    0 18954]]

*******************************

Classification Report : RF
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6155
           1       0.75      1.00      0.86     18954

    accuracy                           0.75     25109
   macro avg       0.38      0.50      0.43     25109
weighted avg       0.57      0.75      0.65     25109


Confusion Matrix : SVM
[[ 3189  2966]
 [ 4460 14494]]

*******************************

Classification Report : SVM

### Analysing Information Gain Models

In [ ]:
run_model(ig_100)

--- Naive Bayes Misclassification ---
--- False Neg Results ---
21115    1
14633    1
42768    1
5054     1
30473    1
dtype: int8
21115    0
14633    0
42768    0
5054     0
30473    0
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
21115,daughter of light,2007-01-01,carole king,Rock,Daughter of light You're a welcome sight To a ...,daughter light youre welcome sight weary soul ...
14633,baby don t you lie to me,2015-01-01,fratellis,Rock,Well little girl you can roll your eyes At the...,well little girl roll eyes sight everything sp...
42768,skin for scars,2007-01-01,cauterize,Rock,Tie these hands they shake too much To hold yo...,tie hands shake much hold hold wandering eyes ...
5054,don t say a word,2003-01-01,crooked fingers,Rock,Don't say a word. Don't tell me what you heard...,dont say word dont tell heard theres something...
30473,new day rising,2011-01-01,the answer,Rock,[Verse 1] All around the sky? s are falling do...,verse around sky falling new kind sound smile...


--- False Pos Results ---
135453    0
126282    0
196460    0
140363    0
155999    0
dtype: int8
135453    1
126282    1
196460    1
140363    1
155999    1
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
135453,wow,2006-01-01,brandy,Pop,No more sittin' at home at night Waiting for y...,sittin home night waiting come home cry found ...
126282,beautiful,2014-01-01,enrique iglesias,Pop,"You, you are so beautiful It used to be, you a...",beautiful used love like world around could fa...
196460,brave face,2007-01-01,delta goodrem,Pop,Put your brave face on The one you wore when y...,put brave face one wore stole heart wont hold ...
140363,why oh why,2006-01-01,atc,Pop,Do da n de da n dai (Do da n de da n dai) It's...,da n de da n dai da n de da n dai feeling got ...
155999,la hallucinations,2015-01-01,carly rae jepsen,Pop,I remember being naked We were young freaks ju...,remember naked young freaks fresh la never car...


--- Random Forest Misclassification ---
--- False Pos Results ---
24002     0
46011     0
135453    0
1621      0
87593     0
dtype: int8
24002     1
46011     1
135453    1
1621      1
87593     1
dtype: int8


,song,year,artist,genre,lyrics,filtered_lyrics
24002,henry,2010-01-01,emma dean,Pop,Aching to be in the room 'cos no one can tell ...,aching room cos one tell everythings going way...
46011,i need a disguise,2014-01-01,belinda carlisle,Pop,There's a mask on the wall That I should be we...,theres mask wall wearing keep seeing im really...
135453,wow,2006-01-01,brandy,Pop,No more sittin' at home at night Waiting for y...,sittin home night waiting come home cry found ...
1621,the homes of donegal,2009-01-01,celtic thunder,Pop,I just dropped in to see you all I'll only sta...,dropped see ill stay awhile want hear youre ge...
87593,time of year,2006-01-01,ashanti,Pop,Christmas is the time of year That we become a...,christmas time year become one tears joy heart...


In [ ]:
# rock_pop_dataset.iloc[126282]

In [35]:
run_model(ig_200)


Confusion Matrix : NB
[[ 2267  3888]
 [ 2603 16351]]

*******************************

Classification Report : NB
              precision    recall  f1-score   support

           0       0.47      0.37      0.41      6155
           1       0.81      0.86      0.83     18954

    accuracy                           0.74     25109
   macro avg       0.64      0.62      0.62     25109
weighted avg       0.72      0.74      0.73     25109


Confusion Matrix : RF
[[    0  6155]
 [    0 18954]]

*******************************

Classification Report : RF
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6155
           1       0.75      1.00      0.86     18954

    accuracy                           0.75     25109
   macro avg       0.38      0.50      0.43     25109
weighted avg       0.57      0.75      0.65     25109


Confusion Matrix : SVM
[[ 3147  3008]
 [ 3680 15274]]

*******************************

Classification Report : SVM

In [33]:
run_model(ig_500)


Confusion Matrix : NB
[[ 2740  3415]
 [ 2948 16006]]

*******************************

Classification Report : NB
              precision    recall  f1-score   support

           0       0.48      0.45      0.46      6155
           1       0.82      0.84      0.83     18954

    accuracy                           0.75     25109
   macro avg       0.65      0.64      0.65     25109
weighted avg       0.74      0.75      0.74     25109


Confusion Matrix : RF
[[    0  6155]
 [    0 18954]]

*******************************

Classification Report : RF
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6155
           1       0.75      1.00      0.86     18954

    accuracy                           0.75     25109
   macro avg       0.38      0.50      0.43     25109
weighted avg       0.57      0.75      0.65     25109


Confusion Matrix : SVM
[[ 3257  2898]
 [ 3815 15139]]

*******************************

Classification Report : SVM